# 문서 검색에서의 계층적 인덱스(Hierarchical Indices)

## 개요

이 코드는 문서 검색을 위한 계층적 인덱싱 시스템을 구현하여 문서 수준 요약과 상세 청크라는 두 가지 수준의 인코딩을 사용합니다. 이 접근 방식은 문서 내 관련 섹션을 요약을 통해 먼저 식별한 후, 해당 섹션에서 세부 정보를 추출하여 정보 검색의 효율성과 관련성을 향상시키는 것을 목표로 합니다.

## 동기

기존의 평면적 인덱싱 방식은 큰 문서나 방대한 데이터에 대해 컨텍스트를 놓치거나 불필요한 정보를 반환하는 문제가 있습니다. 계층적 인덱싱은 이 문제를 해결하기 위해 두 단계 검색 시스템을 제공하며, 더 효율적이고 컨텍스트를 고려한 검색이 가능합니다.

## 주요 구성 요소

1. PDF 처리 및 텍스트 청크 분할
2. OpenAI의 GPT-4를 사용한 비동기 문서 요약
3. 문서 수준 요약 및 상세 청크를 위한 FAISS 및 OpenAI 임베딩 벡터 스토어 생성
4. 커스텀 계층적 검색 함수

## 방법 상세 설명

### 문서 전처리 및 인코딩

1. PDF 파일을 로드하고 문서(일반적으로 페이지 단위)로 분할합니다.
2. 각 문서를 GPT-4를 통해 비동기적으로 요약합니다.
3. 원본 문서를 더 작은 상세 청크로 분할합니다.
4. 두 개의 벡터 스토어를 생성합니다:
   - 문서 수준 요약용 벡터 스토어
   - 상세 청크용 벡터 스토어

### 비동기 처리 및 속도 제한

1. 효율성을 높이기 위해 비동기 프로그래밍(`asyncio`)을 사용합니다.
2. API 속도 제한을 처리하기 위해 배치 처리와 지수 백오프 방식을 구현합니다.

### 계층적 검색

`retrieve_hierarchical` 함수는 두 단계 검색을 수행합니다:

1. 먼저 요약 벡터 스토어에서 검색하여 관련 문서 섹션을 식별합니다.
2. 각 관련 요약에 대해 해당 페이지 번호로 필터링하여 상세 청크 벡터 스토어에서 검색을 수행합니다.
3. 이 접근 방식은 세부 정보가 가장 관련성이 높은 문서 섹션에서만 검색되도록 보장합니다.

## 접근 방식의 장점

1. **검색 효율성 향상**: 요약을 먼저 검색함으로써, 모든 상세 청크를 처리하지 않고도 빠르게 관련 문서 섹션을 식별할 수 있습니다.
2. **컨텍스트 보존**: 계층적 접근 방식은 검색된 정보의 넓은 컨텍스트를 유지하는 데 도움이 됩니다.
3. **확장성**: 이 방식은 대규모 문서나 데이터에 대해 특히 유용하며, 평면적 검색 방식이 비효율적이거나 중요한 컨텍스트를 놓치는 문제를 해결합니다.
4. **유연성**: 검색 시 반환되는 요약 및 청크의 수를 조절할 수 있어, 다양한 사용 사례에 맞게 미세 조정이 가능합니다.

## 구현 세부 사항

1. **비동기 프로그래밍**: Python의 `asyncio`를 활용하여 효율적인 I/O 작업 및 API 호출을 수행합니다.
2. **속도 제한 처리**: API 속도 제한을 효과적으로 관리하기 위해 배치 처리와 지수 백오프 방식을 사용합니다.
3. **영구 저장**: 생성된 벡터 스토어를 로컬에 저장하여 불필요한 재계산을 방지합니다.

## 결론

계층적 인덱싱은 특히 크거나 복잡한 문서 집합에 적합한 고급 문서 검색 접근 방식입니다. 고수준 요약과 세부 청크를 모두 활용함으로써 넓은 컨텍스트 이해와 구체적인 정보 검색 간 균형을 제공합니다. 이 방법은 법률 문서 분석, 학술 연구, 대규모 콘텐츠 관리 시스템 등 효율적이고 컨텍스트를 고려한 정보 검색이


<div style="text-align: center;">

<img src="../images/hierarchical_indices.svg" alt="hierarchical_indices" style="width:50%; height:auto;">
</div>

<div style="text-align: center;">

<img src="../images/hierarchical_indices_example.svg" alt="hierarchical_indices" style="width:100%; height:auto;">
</div>

### Import libraries 

In [1]:
import asyncio
import os
import sys
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.chains.summarize.chain import load_summarize_chain
from langchain.docstore.document import Document

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..'))) # Add the parent directory to the path sicnce we work with notebooks
from helper_functions import *
from evaluation.evalute_rag import *
from helper_functions import encode_pdf, encode_from_string

# Load environment variables from a .env file
load_dotenv()

# Set the OpenAI API key environment variable
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

/var/folders/gl/6qlsk6x94klgs54qzsg9q1jm0000gn/T/ipykernel_4545/75755112.py:10: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from helper_functions import *
/opt/anaconda3/envs/kmy_env/lib/python3.12/site-packages/deepeval/__init__.py:49: UserWarning: You are using deepeval version 1.4.5, however version 1.4.7 is available. You should consider upgrading via the "pip install --upgrade deepeval" command.
  warnings.warn(


### Define document path

In [2]:
path = "../data/Understanding_Climate_Change.pdf"

### Function to encode to both summary and chunk levels, sharing the page metadata

In [3]:
async def encode_pdf_hierarchical(path, chunk_size=1000, chunk_overlap=200, is_string=False):
    """
    PDF 책을 계층적 벡터 스토어로 비동기적으로 인코딩합니다. OpenAI 임베딩을 사용하며,
    지수 백오프를 통해 속도 제한을 처리합니다.

    Args:
        path: PDF 파일의 경로.
        chunk_size: 각 텍스트 청크의 원하는 크기.
        chunk_overlap: 연속적인 청크 간의 겹침 크기.
        
    Returns:
        두 개의 FAISS 벡터 스토어를 포함하는 튜플:
        1. 문서 수준 요약 벡터 스토어
        2. 상세 청크 벡터 스토어
    """
    
    # Step 1: PDF 문서 로드
    if not is_string: # 파일일 경우 비동기 처리 -> pdf 파일을 읽고 문서 객체로 변환하는 과정 
        # PDF 경로에서 문서를 로드
        loader = PyPDFLoader(path)
        # 비동기적으로 로드 작업을 수행하여 I/O를 비동기화 => 다른 작업 처리할 수 있도록함 
        documents = await asyncio.to_thread(loader.load) 
    # 파일 경로가 아니면 문자열이기 때문에 직접 분할 
    else:
        # 텍스트 청크 분할기 설정, 텍스트에서 문서를 생성할 때 사용
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap,
            length_function=len,
            is_separator_regex=False,
        )
        # 문자열로 전달된 경우 텍스트를 청크로 분할
        documents = text_splitter.create_documents([path])

    # Step 2: 문서 수준 요약 생성
    # GPT-4를 사용하여 요약 체인을 설정
    summary_llm = ChatOpenAI(temperature=0, model_name="gpt-4o-mini", max_tokens=4000)
    summary_chain = load_summarize_chain(summary_llm, chain_type="map_reduce")
    

    # 문서를 넣으면 요약하는 함수 
    async def summarize_doc(doc):
        """
        단일 문서를 요약하며 속도 제한을 처리합니다.
        
        Args:
            doc: 요약할 문서.
            
        Returns:
            요약된 Document 객체.
        """
        # 지수 백오프를 통해 요약 호출을 재시도
        summary_output = await retry_with_exponential_backoff(summary_chain.ainvoke([doc]))
        summary = summary_output['output_text']
        return Document(
            page_content=summary,
            metadata={"source": path, "page": doc.metadata["page"], "summary": True}
        )

    # Step 3: 문서들을 작은 배치로 처리하여 속도 제한 방지 -> 요약한 내용을 
    batch_size = 5  # API 속도 제한에 따라 조정 가능
    summaries = []
    # 문서를 배치로 나누어 요약을 수행
    for i in range(0, len(documents), batch_size):
        batch = documents[i:i+batch_size]
        # 각 문서에 대해 요약 작업을 비동기로 수행
        batch_summaries = await asyncio.gather(*[summarize_doc(doc) for doc in batch])
        summaries.extend(batch_summaries)
        await asyncio.sleep(1)  # 각 배치 간에 잠시 대기

    # Step 4: 상세 청크로 문서 분할
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, chunk_overlap=chunk_overlap, length_function=len
    )
    # 문서를 비동기적으로 청크로 분할
    detailed_chunks = await asyncio.to_thread(text_splitter.split_documents, documents)



    # Step 5: 상세 청크에 메타데이터 업데이트
    for i, chunk in enumerate(detailed_chunks):
        # 각 청크에 청크 ID, 요약 여부, 페이지 번호 등 메타데이터 추가 -> 요약하지 않음 
        chunk.metadata.update({
            "chunk_id": i,
            "summary": False,
            "page": int(chunk.metadata.get("page", 0))
        })

    # Step 6: 임베딩 생성
    embeddings = OpenAIEmbeddings()

    # 벡터 스토어 생성 함수 정의 -> 요약이나 상세 청크 분할한 것을 벡터 스토어에 넣음 
    async def create_vectorstore(docs):
        """
        주어진 문서 목록으로부터 벡터 스토어를 생성하며 속도 제한을 처리합니다.
        
        Args:
            docs: 임베딩할 문서 목록.
            
        Returns:
            임베딩된 문서들이 포함된 FAISS 벡터 스토어.
        """
        # 지수 백오프를 통해 벡터 스토어 생성 호출을 재시도
        return await retry_with_exponential_backoff(
            asyncio.to_thread(FAISS.from_documents, docs, embeddings)
        )

    # Step 7: 요약 및 상세 청크에 대한 벡터 스토어를 비동기로 동시에 생성
    summary_vectorstore, detailed_vectorstore = await asyncio.gather(
        create_vectorstore(summaries),
        create_vectorstore(detailed_chunks)
    )

    # 요약 벡터 스토어와 상세 청크 벡터 스토어 반환
    return summary_vectorstore, detailed_vectorstore


##### 요약한 문서 -> 페이지 존재 , 디테일 청크 -> 페이지 넘버 존재 
##### 그러므로, 요약한 문서를 바탕으로 먼저 리트리버하고, 해당 페이지 넘버와 일치하는 디테일 청크를 뽑아내는 방식 

### 요약과 디테일 청크 반환

In [4]:
# pdf를 요약 벡터 스토어, detail 벡터 스토어로 만듦 
if os.path.exists("../vector_stores/summary_store") and os.path.exists("../vector_stores/detailed_store"):
   embeddings = OpenAIEmbeddings()
   summary_store = FAISS.load_local("../vector_stores/summary_store", embeddings, allow_dangerous_deserialization=True)
   detailed_store = FAISS.load_local("../vector_stores/detailed_store", embeddings, allow_dangerous_deserialization=True)

else:
    summary_store, detailed_store = await encode_pdf_hierarchical(path)
    summary_store.save_local("../vector_stores/summary_store")
    detailed_store.save_local("../vector_stores/detailed_store")


### Retrieve information according to summary level, and then retrieve information from the chunk level vector store and filter according to the summary level pages

In [5]:
# 요약에서 topk만큼 쿼리와 유사한 것 뽑고
# 뽑은 것에 해당하는 pdf 페이지 넘버 받은 후에
# detail chunk에서 topk만큼 유사한 것 뽑기 
def retrieve_hierarchical(query, summary_vectorstore, detailed_vectorstore, k_summaries=3, k_chunks=5):

    # 
    top_summaries = summary_vectorstore.similarity_search(query, k=k_summaries)
    
    relevant_chunks = []
    for summary in top_summaries:
        # For each summary, retrieve relevant detailed chunks
        page_number = summary.metadata["page"]
        page_filter = lambda metadata: metadata["page"] == page_number
        page_chunks = detailed_vectorstore.similarity_search(
            query, 
            k=k_chunks, 
            filter=page_filter
        )
        relevant_chunks.extend(page_chunks)
    
    return relevant_chunks

### Demonstrate on a use case

In [6]:
query = "What is the greenhouse effect?"
results = retrieve_hierarchical(query, summary_store, detailed_store)

# Print results
for chunk in results:
    print(f"Page: {chunk.metadata['page']}")
    print(f"Content: {chunk.page_content}...")  # Print first 100 characters
    print("---")

Page: 0
Content: driven by human activities, particularly the emission of greenhou se gases.  
Chapter 2: Causes of Climate Change  
Greenhouse Gases  
The primary cause of recent climate change is the increase in greenhouse gases in the 
atmosphere. Greenhouse gases, such as carbon dioxide (CO2), methane (CH4), and nitrous 
oxide (N2O), trap heat from the sun, creating a "greenhouse effect." This effect is  essential 
for life on Earth, as it keeps the planet warm enough to support life. However, human 
activities have intensified this natural process, leading to a warmer climate.  
Fossil Fuels  
Burning fossil fuels for energy releases large amounts of CO2. This includes coal, oil, and 
natural gas used for electricity, heating, and transportation. The industrial revolution marked 
the beginning of a significant increase in fossil fuel consumption, which continues to rise 
today.  
Coal...
---
Page: 0
Content: Most of these climate changes are attributed to very small variations in 

In [7]:
results

[Document(metadata={'source': '../data/Understanding_Climate_Change.pdf', 'page': 0, 'chunk_id': 2, 'summary': False}, page_content='driven by human activities, particularly the emission of greenhou se gases.  \nChapter 2: Causes of Climate Change  \nGreenhouse Gases  \nThe primary cause of recent climate change is the increase in greenhouse gases in the \natmosphere. Greenhouse gases, such as carbon dioxide (CO2), methane (CH4), and nitrous \noxide (N2O), trap heat from the sun, creating a "greenhouse effect." This effect is  essential \nfor life on Earth, as it keeps the planet warm enough to support life. However, human \nactivities have intensified this natural process, leading to a warmer climate.  \nFossil Fuels  \nBurning fossil fuels for energy releases large amounts of CO2. This includes coal, oil, and \nnatural gas used for electricity, heating, and transportation. The industrial revolution marked \nthe beginning of a significant increase in fossil fuel consumption, which con